In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_cluster import knn_graph
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
import numpy as np

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import tensorflow as tf
import pandas as pd
#from source import Dataset as ds

2024-02-09 11:27:03.913787: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 11:27:03.913873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 11:27:04.180278: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-09 11:27:04.766200: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-09 11:27:15.872217: W tensorflow/comp

In [5]:
a = torch.Tensor(np.arange(10))
b = torch.Tensor(np.arange(10,20))

In [8]:
a = torch.reshape(a, (2,5))
b = torch.reshape(b, (2,5))

In [12]:
torch.cat((a,b), dim = 1)

tensor([[ 0.,  1.,  2.,  3.,  4., 10., 11., 12., 13., 14.],
        [ 5.,  6.,  7.,  8.,  9., 15., 16., 17., 18., 19.]])

In [8]:
#read data
path = '/beegfs/desy/group/unihh2/cms-llp/run3/v10/ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV/dataframes/'
alldf = []
def process_file(i):
    store = pd.HDFStore(path + 'output_' + str(i) + ".h5")
    dfnew = store.select("df",start=0,stop=-1)
    store.close()
    return dfnew


In [9]:
frames = [process_file(j) for j in range(10)]

In [10]:
branches_rechits = [
"cscClusteredRechitsEta",
"cscClusteredRechitsPhi",
"cscClusteredRechitsX",
"cscClusteredRechitsY",
"cscClusteredRechitsZ",
"cscClusteredRechitsTpeak",
"cscClusteredRechitsTwire",
"cscClusteredRechitsChamber",
"cscClusteredRechitsStation",
"cscClusteredRechitsIndex",]

energy_key = "cscRechitCluster_match_gLLP_e"

In [11]:
# Check GPU availability

print("Num GPUs available: ", len(tf.config.list_physical_devices('GPU')), 
      "\nCuda available: ", torch.cuda.is_available(),
     "\nCuda device count: ", torch.cuda.device_count(),
     "\ncurrent device: ", torch.cuda.current_device(),
     "\ndevice info: ", torch.cuda.device(0), 
      "\ndevice name: ", torch.cuda.get_device_name(0))

Num GPUs available:  2 
Cuda available:  True 
Cuda device count:  2 
current device:  0 
device info:  <torch.cuda.device object at 0x2b3384f22070> 
device name:  NVIDIA RTX A6000


In [20]:
# Iterate through all clusters
cluster_features = ["cscRechitClusterX",
                    "cscRechitClusterY",
                    "cscRechitClusterZ",
                    "cscRechitClusterTimeWeighted",
                    "cscRechitClusterTimeSpreadWeightedAll",
                    "cscRechitClusterTime",
                    "cscRechitClusterTimeSpread",
                    "cscRechitClusternXY",
                    "cscRechitClusternZ",
                    "cscRechitClusterXSpread",
                    "cscRechitClusterYSpread",
                    "cscRechitClusterZSpread",
                    "cscRechitClusterXYSpread",
                    "cscRechitClusterRSpread",
                    "cscRechitClusterEtaPhiSpread",
                    "cscRechitClusterEtaSpread",
                    "cscRechitClusterPhiSpread",
                    "cscRechitClusterDeltaRSpread",
                    "cscRechitClusterMajorAxis",
                    "cscRechitClusterMinorAxis",
                    "cscRechitClusterSkewX",
                    "cscRechitClusterSkewY",
                    "cscRechitClusterSkewZ",
                    "cscRechitClusterKurtX",
                    "cscRechitClusterKurtY",
                    "cscRechitClusterKurtZ",
                    "cscRechitClusterPhi",
                    "cscRechitClusterEta"]

def make_graphs(df_signal):
    num_edges = 8
    node_features = []
    graph_features = []
    edges = []
    label_e = []
    for i in range(df_signal.index[-1][0]):
        # graph level
        graph_level = []
        for feature in cluster_features:
            graph_level.append(df_signal[feature][i][0])
            
        # features for edge making
        nodes_edge = torch.Tensor(np.transpose([df_signal['cscClusteredRechitsRelEta'][i].to_numpy(),
                                                df_signal['cscClusteredRechitsRelPhi'][i].to_numpy()]))
        # features for nodes
        nodes = torch.Tensor(np.transpose([df_signal['cscClusteredRechitsRelEta'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsRelPhi'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsRelX'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsRelY'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsRelZ'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsTpeak'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsTwire'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsStation'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsChamber'][i].to_numpy()]))
        
        graph_features.append(torch.Tensor(graph_level))
        node_features.append(nodes)
        edges.append(knn_graph(nodes_edge, num_edges))
        label_e.append(torch.Tensor([df_signal[energy_key][i][0]]))

    print(len(node_features), len(edges), len(label_e), len(graph_features))
    graph_data = []
    for i in range(len(node_features)):
        graph_data.append(Data(x=node_features[i], edge_index=edges[i], u=graph_features[i], y=label_e[i]))
    return graph_data

In [21]:
# make list of graphs in Data format
out_data = []
for frame in frames:
    out_data.extend(make_graphs(frame))

983 983 983 983
967 967 967 967
958 958 958 958
951 951 951 951
936 936 936 936
901 901 901 901
962 962 962 962
1033 1033 1033 1033
962 962 962 962
933 933 933 933


In [25]:
# Save finished graphs here

torch.save(out_data, '/beegfs/desy/group/unihh2/cms-llp/run3/v10/ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV/graphs/graphs9k8nn_' + str(1) + '.pt')

In [28]:
a = out_data[0].u
b = out_data[1].u

In [32]:
c = None
torch.cat((a,b), 0)

tensor([-1.3881e+02,  6.3269e+02, -8.4082e+02,  5.1147e+00,  6.5854e+00,
         5.7381e+00,  6.5359e+00,  5.5000e+01,  5.5000e+01,  2.2584e+01,
         3.1850e+01,  1.3738e-01,  1.3890e+01,  3.2774e+01,  1.1482e-02,
         4.0626e-02,  3.2724e-02,  5.2166e-02,  4.0976e-02,  3.2285e-02,
        -2.6505e+00, -1.4029e+01,  6.0668e-02, -1.0588e+00, -1.4606e+00,
        -1.4705e+00,  1.7868e+00, -1.0773e+00,  1.6385e+02,  1.6618e+02,
         7.3625e+02, -3.3163e+01,  1.2569e+01, -3.4912e+01,  1.2563e+01,
         5.7000e+01,  5.7000e+01,  4.5193e+01,  3.1421e+01,  1.5978e+02,
         3.4995e+01,  5.2464e+01,  1.8093e-02,  6.8463e-02,  7.0287e-02,
         9.8120e-02,  7.1866e-02,  6.6804e-02,  3.0733e+01,  8.6755e+00,
         6.1625e+01, -9.3486e-01, -1.1879e+00, -1.4595e+00,  7.9247e-01,
         1.8663e+00])

In [33]:
a

tensor([-1.3881e+02,  6.3269e+02, -8.4082e+02,  5.1147e+00,  6.5854e+00,
         5.7381e+00,  6.5359e+00,  5.5000e+01,  5.5000e+01,  2.2584e+01,
         3.1850e+01,  1.3738e-01,  1.3890e+01,  3.2774e+01,  1.1482e-02,
         4.0626e-02,  3.2724e-02,  5.2166e-02,  4.0976e-02,  3.2285e-02,
        -2.6505e+00, -1.4029e+01,  6.0668e-02, -1.0588e+00, -1.4606e+00,
        -1.4705e+00,  1.7868e+00, -1.0773e+00])

In [34]:
b

tensor([ 1.6385e+02,  1.6618e+02,  7.3625e+02, -3.3163e+01,  1.2569e+01,
        -3.4912e+01,  1.2563e+01,  5.7000e+01,  5.7000e+01,  4.5193e+01,
         3.1421e+01,  1.5978e+02,  3.4995e+01,  5.2464e+01,  1.8093e-02,
         6.8463e-02,  7.0287e-02,  9.8120e-02,  7.1866e-02,  6.6804e-02,
         3.0733e+01,  8.6755e+00,  6.1625e+01, -9.3486e-01, -1.1879e+00,
        -1.4595e+00,  7.9247e-01,  1.8663e+00])